<a href="https://colab.research.google.com/github/aurotripathy/RL-Experiments/blob/main/assistant_mcp_sqlite_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###  A sqlite database assistant implemented by Qwen Agent
From, below...getting it working on colab

https://github.com/QwenLM/Qwen-Agent/blob/main/examples/assistant_mcp_sqlite_bot.py

In [ ]:
!pip install -U "qwen-agent[gui,rag,code_interpreter,mcp]"

In [ ]:
"""A sqlite database assistant implemented by assistant"""

import os
import asyncio
from typing import Optional

from qwen_agent.agents import Assistant
from qwen_agent.gui import WebUI

ROOT_RESOURCE = os.path.join(os.getcwd(), 'resource')

In [ ]:
def init_agent_service():
    llm_cfg = {'model': 'qwen-max'}
    system = ('You play the role of a database assistant; you have the ability to query databases.')
    tools = [{
        "mcpServers": {
            "sqlite" : {
                "command": "uvx",
                "args": [
                    "mcp-server-sqlite",
                    "--db-path",
                    "test.db"
                ]
            }
        }
    }]
    bot = Assistant(
        llm=llm_cfg,
        name='数据库助手',
        description='数据库查询',
        system_message=system,
        function_list=tools,
    )

    return bot




In [ ]:
def test(query='数据库里有几张表', file: Optional[str] = os.path.join(ROOT_RESOURCE, 'poem.pdf')):
    # Define the agent
    bot = init_agent_service()

    # Chat
    messages = []

    if not file:
        messages.append({'role': 'user', 'content': query})
    else:
        messages.append({'role': 'user', 'content': [{'text': query}, {'file': file}]})

    for response in bot.run(messages):
        print('bot response:', response)


def app_tui():
    # Define the agent
    bot = init_agent_service()

    # Chat
    messages = []
    while True:
        # Query example: 数据库里有几张表
        query = input('user question: ')
        # File example: resource/poem.pdf
        file = input('file url (press enter if no file): ').strip()
        if not query:
            print('user question cannot be empty！')
            continue
        if not file:
            messages.append({'role': 'user', 'content': query})
        else:
            messages.append({'role': 'user', 'content': [{'text': query}, {'file': file}]})

        response = []
        for response in bot.run(messages):
            print('bot response:', response)
        messages.extend(response)


def app_gui():
    # Define the agent
    bot = init_agent_service()
    chatbot_config = {
        'prompt.suggestions': [
            'How many tables are there in the database?',
            "Create a student table that includes the student's name and age."
            'Add a student named Han Meimei, who is 6 years old this year.',
        ]
    }
    WebUI(
        bot,
        chatbot_config=chatbot_config,
    ).run()




In [ ]:
if __name__ == '__main__':
    # test()
    # app_tui()
    app_gui()